In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/urdu-articles-raw/urdu_wikipedia_articles.json


In [ ]:
import json

# Define the file path
file_path = '/kaggle/input/urdu-articles-raw/urdu_wikipedia_articles.json'

# Load the JSON data
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Print the first few entries (if it's a list or dict)
# print(data[:5] if isinstance(data, list) else data)


In [ ]:
data[0]

In [ ]:
content_count = 0

# Assuming 'data' is a list of articles
for entry in data:
    if 'content' in entry:  # Check if 'content' key exists
        content_count += 1  # Increment the count

print(f"Number of 'content' objects: {content_count}")

In [4]:
import re
import unicodedata

class TextCleaner:
    def __init__(self):
        # Urdu normalization dictionary
        self.urdu_normalization_dict = {
            'أ': 'ا', 'إ': 'ا', 'ٱ': 'ا',
            'ٻ': 'ب', 'ة': 'ت', 'ۃ': 'ت',
            'ٿ': 'ث', 'ڃ': 'ج', 'ځ': 'ح',
            'ڊ': 'د', 'ڒ': 'ر', 'ڛ': 'س',
            'ڜ': 'ش', 'ڝ': 'ص', 'ڠ': 'ع',
            'ڧ': 'غ', 'ڤ': 'ف', 'ڥ': 'ف', 'ڦ': 'ف',
            'ڨ': 'ق', 'ڪ': 'ک', 'ڰ': 'گ', 'ڱ': 'گ',
            'ڵ': 'ل', 'ݥ': 'م', 'ۆ': 'و', 'ۇ': 'و',
            'ۈ': 'و', 'ۉ': 'و', 'ۊ': 'و', 'ۋ': 'و',
            'ٔ': 'ء', 'ﺀ': 'ء',
            '\u064B': '', '\u064C': '', '\u064D': '', 
            '\u064E': '', '\u064F': '', '\u0650': '', 
            '\u0651': '', '\u0652': '', '\u0653': '', 
            '\u0656': '', '\u0657': '', '\u0658': '', 
            '\u0670': '',
        }

    def remove_non_urdu(self, text):
        urdu_range = r'\u0600-\u06FF'
        basic_punctuations = r'۔،؛؟!'
        urdu_digits = r'۰-۹'
        allowed_chars = urdu_range + urdu_digits + basic_punctuations
        return re.sub(f'[^{allowed_chars}\s]', '', text)

    def normalize_unicode(self, text):
        return unicodedata.normalize('NFKC', text)

    def normalize_urdu_specific(self, text):
        pattern = re.compile('|'.join(map(re.escape, self.urdu_normalization_dict.keys())))
        text = pattern.sub(lambda m: self.urdu_normalization_dict[m.group(0)], text)
        text = re.sub(r'([اآءٰٓؤى])ا', r'\1', text)
        text = text.replace('ؤ', 'و')
        text = re.sub(r'\s*([۔،؛؟!])\s*', r' \1 ', text)
        return text.strip()

    def remove_extra_spaces(self, text):
        return re.sub(r'\s+', ' ', text).strip()

    def remove_exact_duplicates(self, text):
        sentences = re.split(r'(?<=[.!؟])\s+', text.strip())
        seen = set()
        unique_sentences = []

        for sentence in sentences:
            normalized_sentence = self.normalize_urdu_specific(sentence)
            if normalized_sentence and normalized_sentence not in seen:
                seen.add(normalized_sentence)
                unique_sentences.append(sentence)

        return ' '.join(unique_sentences)

    def clean(self, text):
        text = self.normalize_unicode(text)
        text = self.remove_non_urdu(text)
        text = self.normalize_urdu_specific(text)
        text = self.remove_extra_spaces(text)
        text = self.remove_exact_duplicates(text)
        return text

    def clean_sentences(self, text):
        cleaned_text = self.clean(text)
        sentences = re.split(r'(?<=[۔!?])\s*', cleaned_text)
        return [s for s in sentences if s]  # Remove empty strings



In [ ]:
if __name__ == "__main__":
    cleaner = TextCleaner()
    
    
    text = ""    
    text = cleaner.clean(text)
    print(text)


In [ ]:
# Usage example
if __name__ == "__main__":
    cleaner = TextCleaner()
    
    # Extract and clean content from data
    cleaned_articles = [cleaner.clean_sentences(entry['content']) for entry in data]

    # Flatten the list of sentences for easier handling
    all_sentences = [sentence for sentences in cleaned_articles for sentence in sentences]

    # Optionally, print some cleaned sentences
    for i, cleaned_sentence in enumerate(all_sentences[:5]):  # Print the first 5 cleaned sentences
        print(f"Cleaned Sentence {i+1}: {cleaned_sentence}")

In [ ]:
count = len(all_sentences)
print(count)

In [ ]:
# Write all sentences to a .txt file
with open("cleaned_sentences.txt", "w", encoding="utf-8") as f:
    for sentence in all_sentences:
        f.write(sentence + "\n")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        # Ensure that the model dimension (d_model) is divisible by the number of heads
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        # Initialize dimensions
        self.d_model = d_model # Model's dimension
        self.num_heads = num_heads # Number of attention heads
        self.d_k = d_model // num_heads # Dimension of each head's key, query, and value
        
        # Linear layers for transforming inputs
        self.W_q = nn.Linear(d_model, d_model) # Query transformation
        self.W_k = nn.Linear(d_model, d_model) # Key transformation
        self.W_v = nn.Linear(d_model, d_model) # Value transformation
        self.W_o = nn.Linear(d_model, d_model) # Output transformation
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        # Calculate attention scores
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # Apply mask if provided (useful for preventing attention to certain parts like padding)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e4)
        
        # Softmax is applied to obtain attention probabilities
        attn_probs = torch.softmax(attn_scores, dim=-1)
        
        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        # Reshape the input to have num_heads for multi-head attention
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        # Combine the multiple heads back to original shape
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        # Apply linear transformations and split heads
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        # Perform scaled dot-product attention
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        
        # Combine heads and apply output transformation
        output = self.W_o(self.combine_heads(attn_output))
        return output

    
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.fc2(self.dropout(self.relu(self.fc1(x))))
    
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]
    
    
    
    
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
    
    
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [4]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

In [6]:
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length))  # (batch_size, seq_length)

In [ ]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

In [ ]:
!pip install python-docx

In [7]:
from docx import Document

# Specify the path to the .txt file
txt_file_path = '/kaggle/input/clean-urdu-wiki/cleaned_sentences_wiki.txt'
docx_file_path = 'cleaned_sentences_wiki.docx'

# Create a new Document
doc = Document()

# Read and add paragraphs in a more efficient way
with open(txt_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        doc.add_paragraph(line.strip())

# Save the .docx file
doc.save(docx_file_path)

print(f'Document created: {docx_file_path}')


ModuleNotFoundError: No module named 'docx'